In [7]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [8]:
# Устанавливаем значение рабочей дирректории.
os.chdir("C:/Users/Michael/Documents/DataSet/Koronastat")

In [9]:
# Чтение данных из таблицы.
Koronastat = pd.read_csv("Таблица_2021-10-29_21-42.csv", delimiter=',')

In [10]:
# Выводим на экран первые 5 элементов таблицы.
Koronastat.head()

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
0,20.09.2020,Томская обл.,6775,5554,79,1,62,92
1,26.09.2020,Костромская обл.,4553,3329,67,5,54,44
2,02.09.2021,Ямало-Ненецкий АО,47772,45810,625,4,102,121
3,15.12.2020,Сахалинская обл.,13406,10292,11,0,142,366
4,21.09.2020,Волгоградская обл.,14158,12376,131,0,95,13


In [11]:
# С помощью pandas изменим текст "Дата" на объект даты.
Koronastat["Дата"] = pd.to_datetime(Koronastat["Дата"], format="%d.%m.%Y")
# Удалим из даты время.
Koronastat["Дата"] = Koronastat["Дата"].dt.date
# Определим самую последнюю дату.
most_recent_date = Koronastat["Дата"].max()

In [12]:
# Сохраним в отдельные переменные выборки, которые собираемся тестировать.
Moscow_set = Koronastat[Koronastat["Регион"] == "Москва"]
Sankt_Petersburg_set = Koronastat[Koronastat["Регион"] == "Санкт-Петербург"]

In [13]:
Moscow_set

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
100,2020-06-21,Москва,213946,135556,3597,32,968,1007
160,2021-08-18,Москва,1547244,1417825,26766,59,1590,3029
224,2020-04-11,Москва,8852,499,58,8,1030,149
272,2020-11-04,Москва,445181,325531,7004,68,5826,3852
367,2021-04-04,Москва,1034567,949843,16872,56,1901,998
...,...,...,...,...,...,...,...,...
50318,2020-04-19,Москва,24324,1763,176,28,3570,84
50372,2021-02-16,Москва,959405,873898,14464,60,1409,2062
50525,2021-05-30,Москва,1177356,1060588,20060,59,3719,2547
50604,2020-10-19,Москва,367629,275648,6009,51,5376,1767


In [7]:
# Сохраним в отдельные переменные информацию, за последние сутки.
Moscow_set_last_day = Moscow_set[Moscow_set["Дата"] == most_recent_date]
Sankt_Petersburg_set_last_day = Sankt_Petersburg_set[Sankt_Petersburg_set["Дата"] == most_recent_date]

In [8]:
# Выведем информацию по Covid-19 за последние сутки в Москве
Moscow_set_last_day

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
13900,2021-10-29,Москва,1802521,1583191,31151,93,7511,5134


In [9]:
# Выведем информацию по Covid-19 за последние сутки в Санкт-Петербурге
Sankt_Petersburg_set_last_day

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
43779,2021-10-29,Санкт-Петербург,701379,638284,23722,76,3452,3417


In [10]:
# Сохраним набор количества испытаний/наблюдений по числу заражений за день в Москве и Санкт-Петербурге
number_of_infections = np.array([Moscow_set_last_day["Заражений за день"], Sankt_Petersburg_set_last_day["Заражений за день"]])
# Сохраним набор количества успехов в испытаниях (число смертей за день в Москве и Санкт-Петербурге)
number_of_deaths = np.array([Moscow_set_last_day["Смертей за день"], Sankt_Petersburg_set_last_day["Смертей за день"]])
# Сохраним набор количества успехов в испытаниях (число выздоровлений за день в Москве и Санкт-Петербурге)
number_of_recoveries = np.array([Moscow_set_last_day["Выздоровлений за день"], Sankt_Petersburg_set_last_day["Выздоровлений за день"]])

In [11]:
# Найдём и выведем значение z-метки и p-value.
z_mark, p_value = sm.stats.proportions_ztest(number_of_deaths, number_of_infections)
print(['{:.12f}'.format(i.item()) for i in (z_mark, p_value)])

['-3.803104924579', '0.000142893798']


In [12]:
# Найдём и выведем значение z-метки и p-value.
z_mark, p_value = sm.stats.proportions_ztest(number_of_recoveries, number_of_infections)
print(['{:.12f}'.format(i.item()) for i in (z_mark, p_value)])

['-35.961813961808', '0.000000000000']


Значение p-value, у нас значительно меньше, чем уровень значимости альфа = 0.05, а значит, что нулевую гипотезу опровергаем, значит доли статистически значимо различаются

In [13]:
Kaliningrad_set = Koronastat[Koronastat["Регион"] == "Калининградская обл."]
Kaliningrad_set_last_day = Kaliningrad_set[Kaliningrad_set["Дата"] == most_recent_date]

Chukotka_set = Koronastat[Koronastat["Регион"] == "Чукотский АО"]
Chukotka_set_last_day = Chukotka_set[Chukotka_set["Дата"] == most_recent_date]

number_of_infections = np.array([Kaliningrad_set_last_day["Заражений за день"], Chukotka_set_last_day["Заражений за день"]])
number_of_deaths = np.array([Kaliningrad_set_last_day["Смертей за день"], Chukotka_set_last_day["Смертей за день"]])
number_of_recoveries = np.array([Kaliningrad_set_last_day["Выздоровлений за день"], Chukotka_set_last_day["Выздоровлений за день"]])

# Найдём и выведем значение z-метки и p-value.
z_mark, p_value = sm.stats.proportions_ztest(number_of_deaths, number_of_infections)
print(['{:.12f}'.format(i.item()) for i in (z_mark, p_value)])

# Найдём и выведем значение z-метки и p-value.
z_mark, p_value = sm.stats.proportions_ztest(number_of_recoveries, number_of_infections)
print(['{:.12f}'.format(i.item()) for i in (z_mark, p_value)])

['0.300343369043', '0.763915255579']
['0.126396037547', '0.899418438578']


Значение p-value, у нас больше, чем уровень значимости альфа = 0.05, а значит, что нулевую гипотезу не опровергаем, значит доли статистически не значимо различаются

In [14]:
# Выведем информацию по Covid-19 за последние сутки в Калининградской обл.
Kaliningrad_set_last_day

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
31959,2021-10-29,Калининградская обл.,63842,55722,854,2,401,295


In [15]:
# Выведем информацию по Covid-19 за последние сутки в Чукотской АО
Chukotka_set_last_day

,Дата,Регион,Заражений,Выздоровлений,Смертей,Смертей за день,Заражений за день,Выздоровлений за день
36545,2021-10-29,Чукотский АО,2335,1827,20,0,18,13
